<a href="https://colab.research.google.com/github/KadinRelefourd/neuralnetworkclassifier/blob/main/classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
import torchvision
import torchvision.transforms as transforms
import os
import cv2
import sys

# Device configuration
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print(f"Using device: {device}")

# Data preparation (CIFAR-10 dataset)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.Lambda(lambda x: x.view(-1))  # Flatten to 3072 features (32x32x3)
])

train_data = torchvision.datasets.CIFAR10(
    root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(
    root='./data', train=False, transform=transform)

train_loader = Data.DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = Data.DataLoader(dataset=test_data, batch_size=64)

# Neural network definition
class Net(nn.Module):
    def __init__(self, n_input, n_hidden_1, n_hidden_2, n_output):
        super(Net, self).__init__()
        self.hidden1 = nn.Linear(n_input, n_hidden_1)
        self.hidden2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.out = nn.Linear(n_hidden_2, n_output)
        self.dropout = nn.Dropout(0.5)  # Dropout for regularization

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = self.dropout(x)  # Apply dropout after activation
        x = F.relu(self.hidden2(x))
        x = self.dropout(x)
        return self.out(x)

# Initialize model
model = Net(3072, 512, 256, 10).to(device)  # Input size is 3072 for CIFAR-10 images
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training function
def train():
    for epoch in range(10):  # Train for 10 epochs
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for step, (x, y) in enumerate(train_loader):
            x, y = x.to(device), y.to(device)

            # Forward pass and optimization
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_func(outputs, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted_train = torch.max(outputs.data, 1)
            total_train += y.size(0)
            correct_train += (predicted_train == y).sum().item()

        # Calculate train accuracy and test accuracy/loss after each epoch
        train_acc = 100 * correct_train / total_train
        test_loss, test_acc = test()

        print(f'{epoch+1}/10\t{running_loss/len(train_loader):.4f}\t{train_acc:.4f}\t{test_loss:.4f}\t{test_acc:.4f}')

        save_model()  # Save the model after each epoch

# Testing function
def test():
    model.eval()  # Set model to evaluation mode
    correct_test = 0
    total_test = 0
    running_test_loss = 0.0

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = loss_func(outputs, y)
            running_test_loss += loss.item()

            _, predicted_test = torch.max(outputs.data, 1)
            total_test += y.size(0)
            correct_test += (predicted_test == y).sum().item()

    test_acc = 100 * correct_test / total_test
    return running_test_loss / len(test_loader), test_acc

# Save the trained model to disk
def save_model():
    if not os.path.exists('./model'):
        os.makedirs('./model')
    torch.save(model.state_dict(), './model/model.ckpt')

# Load a trained model from disk
def load_model():
    model.load_state_dict(torch.load('./model/model.ckpt'))
    model.eval()

# Predict the class of a single image file
def predict_image(image_path):
    load_model()

    # Read and preprocess the image using OpenCV and PyTorch transforms
    image = cv2.imread(image_path)
    image = cv2.resize(image, (32, 32))  # Resize to CIFAR-10 dimensions (32x32)
    image_tensor = torch.tensor(image / 255.0).permute(2, 0, 1).float().unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image_tensor.view(-1))
        _, predicted_class_idx = torch.max(outputs.data, 1)

    classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
    return classes[predicted_class_idx.item()]

# Main function to handle train/test commands in Colab or terminal environments
if __name__ == "__main__":
    args_list = sys.argv[1:]  # Get command-line arguments

    if "train" in args_list:
        print("Loop\tTrain Loss\tTrain Acc %\tTest Loss\tTest Acc %")
        train()
    elif "test" in args_list:
        if len(args_list) > args_list.index("test") + 1:
            image_path = args_list[args_list.index("test") + 1]
            print(f'Prediction result: {predict_image(image_path)}')
        else:
            print("Error: Please provide an image path for testing.")


Using device: cuda
Files already downloaded and verified


In [6]:
!pip install torch torchvision opencv-python
!python classify.py train


python3: can't open file '/content/classify.py': [Errno 2] No such file or directory
